In [17]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.corpus import stopwords

In [18]:
conn = create_engine('postgresql://*******@localhost:5432/mcnulty_songs').raw_connection()
cursor = conn.cursor()

In [19]:
stops = list(set(stopwords.words()))

In [20]:
df = pd.DataFrame(stops, columns=['stopword'])
df.set_index('stopword', inplace=True)


In [21]:
df.to_csv('stopwords.csv')

In [22]:
cursor.execute('''CREATE TEMP TABLE stopword (
                    stopword character(30)
                    ) ON COMMIT DROP;''')

with open('stopwords.csv', 'r') as f:
    cmd = 'COPY stopword FROM STDIN WITH (FORMAT CSV, HEADER TRUE)'
    cursor.copy_expert(cmd, f)
    #conn.commit()
    
cursor.execute('''CREATE OR REPLACE VIEW lyrics_without_stop_words AS
                      SELECT track_id, word, count, is_test
                      FROM lyrics WHERE NOT EXISTS (
                          SELECT 1
                          FROM stopword
                          WHERE lyrics.word = stopword.stopword
                      );''')



In [23]:
#3:31
cursor.execute('''INSERT INTO stopless_lyrics
                    SELECT * from lyrics_without_stop_words;''')
conn.commit()

In [ ]:
cursor.execute('''CREATE TABLE stopless_lyrics ( 
                track_id character(18),
                word character(60),
                count integer,
                is_test boolean
               );''')

cursor.execute('''CREATE INDEX track_slyric_index ON stopless_lyrics (track_id);''')
cursor.execute('''CREATE INDEX word_slyric_index ON stopless_lyrics (word);''')
cursor.execute('''CREATE INDEX count_slyric_index ON stopless_lyrics (count);''')
cursor.execute('''CREATE INDEX test_slyric_index ON stopless_lyrics (is_test);''')

conn.commit()